In [1]:
import pandas

%env RAY_OBJECT_STORE_ALLOW_SLOW_STORAGE=1

import modin.pandas as pd 

import modin.config as cfg; print(cfg.CpuCount.get()); cfg.CpuCount.put(8); print(cfg.CpuCount.get())

from datasets import load_dataset

from tqdm import tqdm

from ftfy import fix_text

pd.set_option('max_colwidth', 400)


#Fix seed
seed = 4212


env: RAY_OBJECT_STORE_ALLOW_SLOW_STORAGE=1
16
8


In [2]:
def load_hf_dataset(name, train_slices, val_slices, test_slices, fields, label_dict, extra_hf_field=None):
    """
    Params:
    name: Name of the dataset on HF
    train_slices: 
    """
    if extra_hf_field:
        dataset = load_dataset(name, extra_hf_field)
    else:
        dataset = load_dataset(name)
        
    name = name if not extra_hf_field else name+"_"+extra_hf_field
    
    print("Dataset Name: ", name)    
    print(dataset)
    
    label_field = fields[2]
    
    train = pd.DataFrame()
    val = pd.DataFrame()
    test = pd.DataFrame()
    
    if train_slices is not None:
        for s in train_slices:
            temp = pd.DataFrame(dataset[s][:])[fields]
            train = pd.concat([train, temp], ignore_index=True)
            # train = train.append(temp)
        train['D'] = name
        train = train.rename(columns = {label_field: 'L', fields[0]: 'P', fields[1]: 'H'})
        train['L'] = train['L'].astype(str).replace(label_dict)
        print("Train: ", len(train))
    
    if val_slices is not None:
        for s in val_slices:
            temp = pd.DataFrame(dataset[s][:])[fields]
            val = pd.concat([val, temp], ignore_index=True)
            # val = val.append(temp)               
        val['D'] = name
        val = val.rename(columns = {label_field: 'L', fields[0]: 'P', fields[1]: 'H'})
        val['L'] = val['L'].astype(str).replace(label_dict)
        print("Val: ", len(val))

        
    if test_slices is not None:
        for s in test_slices:
            temp = pd.DataFrame(dataset[s][:])[fields]
            test = pd.concat([test, temp], ignore_index=True)
            # test = test.append(temp)
        test['D'] = name
        test = test.rename(columns = {label_field: 'L', fields[0]: 'P', fields[1]: 'H'})
        test['L'] = test['L'].astype(str).replace(label_dict)
        print("Test: ", len(test))
        
    return train, val, test   

def append_to_global(train, val, test):
    global all_train, all_val, all_test
    
    if train is not None:
        # all_train = all_train.append(train, ignore_index=True)
        all_train = pd.concat([all_train, train], ignore_index=True)
        
    if val is not None:
        # all_val = all_val.append(val, ignore_index=True)
        all_val = pd.concat([all_val, val], ignore_index=True)
        
    if test is not None:
        # all_test = all_test.append(test, ignore_index=True)
        all_test = pd.concat([all_test, test], ignore_index=True)

In [3]:
all_train = pd.DataFrame()
all_val = pd.DataFrame()
all_test = pd.DataFrame()

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2024-02-07 10:38:53,981	INFO worker.py:1724 -- Started a local Ray instance.


In [4]:
# #ANLI 

# t1,t2,t3 = load_hf_dataset("anli",
#                     ["train_r3"],
#                     ["dev_r1","dev_r2","dev_r3"],
#                     ["test_r1","test_r2","test_r3"],
#                     ['premise','hypothesis','label'],
#                     {'0': 'e', '1': 'n', '2': 'c'},
#                    )

# t1_r1_r2,_,_ = load_hf_dataset("anli",
#                     ["train_r1", "train_r2"],
#                     None,
#                     None,
#                     ['premise','hypothesis','label'],
#                     {'0': 'e', '1': 'n', '2': 'c'},
#                    )


# count = 0
# def get_length(row):
#     global count
#     text_a = row['P']
#     text_a = len(text_a.split())
    
#     row['len(P)'] = text_a
    
#     if not count % 1000:
#         print(count)
#     count+=1
    
#     return row


# t1 = t1.apply(get_length, axis=1)
# t1_r1_r2_1 = t1_r1_r2[t1_r1_r2['L'] == "e"]
# t1_r1_r2_2 = t1_r1_r2[t1_r1_r2['L'] != "e"].sample(len(t1_r1_r2_1) - 10000, random_state=seed)

# t1_r1_r2 = pd.concat([t1_r1_r2_1, t1_r1_r2_2])

# tmp = t1[t1['L'] != "e"]
# tmp1 = tmp[tmp["len(P)"] < 15].drop_duplicates(subset=["P"], keep="first")
# tmp2 = tmp[tmp["len(P)"] > 15]

# t1_new_entail = t1[t1['L'] == "e"]
# t1_new_contradict = pd.concat([tmp1, tmp2])
 
# t1_new_contradict = t1_new_contradict[["P","H","L","D"]]
# t1_new_contradict = t1_new_contradict.sample(len(t1_new_entail), random_state=seed)

# t1_new_entail = t1_new_entail[['P','H','L','D']]

# t1_new = pd.concat([t1_r1_r2, t1_new_entail, t1_new_contradict])


# append_to_global(t1_new, t2, t3)

In [5]:
#ANLI 

t1,t2,t3 = load_hf_dataset("anli",
                    ["train_r1", "train_r2", "train_r3"],
                    ["dev_r1","dev_r2","dev_r3"],
                    ["test_r1","test_r2","test_r3"],
                    ['premise','hypothesis','label'],
                    {'0': 'e', '1': 'n', '2': 'c'},
                   )


train_entail = t1[t1['L'] == "e"]
train_contradict = t1[t1['L'] == "c"]
train_neutral = t1[t1['L'] == "n"].sample(11000, random_state=seed)
t1 = pd.concat([train_entail, train_contradict, train_neutral], ignore_index=True)

append_to_global(t1, t2, t3)

t1.value_counts('L')

Dataset Name:  anli
DatasetDict({
    train_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 16946
    })
    dev_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    test_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    train_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 45460
    })
    dev_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    test_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    train_r3: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 100459
    })
    dev_r3: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],


Train:  162865
Val:  3200
Test:  3200


the groupby keys will be sorted anyway, although the 'sort=False' was passed. See the following issue for more details: https://github.com/modin-project/modin/issues/3571.


L
e    52111
c    41965
n    11000
Name: count, dtype: int64

In [6]:
#WANLI
t1,t2,t3 = load_hf_dataset("alisawuffles/WANLI",
                    ["train"],
                    ["test"],
                    None,
                    ['premise','hypothesis','gold'],
                    {'entailment': 'e', 'neutral': 'n', 'contradiction': 'c'},
                   )


# Increase the number of entailment examples in the training set
x = t1[t1['L'] == "e"].sample(25000, random_state=seed)
train = pd.concat([t1, x], ignore_index=True)


append_to_global(train, None, t2)

train.value_counts('L')

Dataset Name:  alisawuffles/WANLI
DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'gold', 'genre', 'pairID'],
        num_rows: 102885
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'gold', 'genre', 'pairID'],
        num_rows: 5000
    })
})


Train:  102885
Val:  5000


L
e    63511
n    48977
c    15397
Name: count, dtype: int64

In [7]:
# #QQP:  Subsample 10k longest pairs from this dataset 
# t1,t2,t3 = load_hf_dataset("glue",
#                     ["train"],
#                     ["validation"],
#                     None,
#                     ['question1','question2','label'],
#                     {'1': 'e', '0': 'n'},
#                     "qqp"
#                    )


# t1 = t1.apply(get_length, axis=1)
# tmp = t1[t1["len(P)"] > 19]
# tmp1 = tmp[tmp["L"] == "e"].sample(n=3000, random_state=seed)
# tmp2 = tmp[tmp["L"] != "e"].sample(n=3000, random_state=seed)

# t1_new = pd.concat([tmp1, tmp2])[["P","H","L","D"]]

# append_to_global(t1_new, None, None)


In [8]:
#RTE
t1,t2,t3 = load_hf_dataset("glue",
                    ["train"],
                    ["validation"],
                    None,
                    ['sentence1','sentence2','label'],
                    {'0': 'e', '1': 'n'},
                    "rte"
                   )

append_to_global(t1, t2, None)

t1.value_counts('L')

Dataset Name:  glue_rte
DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})
Train:  2490
Val:  277


L
e    1249
n    1241
Name: count, dtype: int64

In [9]:
#SciTail
t1,t2,t3 = load_hf_dataset("scitail",
                    ["train"],
                    ["validation"],
                    ["test"],
                    ['premise','hypothesis','label'],
                    {'entails': 'e', 'neutral': 'n'},
                    "tsv_format"
                   )


t1_contradict = t1[t1['L'] != "e"]
t1_entail = t1[t1['L'] == "e"]

t1_contradict = t1_contradict.sample(len(t1_entail), random_state=seed)
t1 = pd.concat([t1_contradict, t1_entail], ignore_index=True)
t1['D'] = "scitail"
t2['D'] = "scitail"
t3['D'] = "scitail"

append_to_global(t1, t2, t3)

t1.value_counts('L')

Dataset Name:  scitail_tsv_format
DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 23097
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2126
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 1304
    })
})
Train:  23097
Val:  1304
Test:  2126


L
e    8472
n    8472
Name: count, dtype: int64

In [10]:
#LingNLI
t1 = pandas.read_json("https://raw.githubusercontent.com/Alicia-Parrish/ling_in_loop/master/NLI_data/3_Ling_in_loop_protocol/train_round5_LitL_combined.jsonl", lines=True)[['premise','hypothesis','label']]
t1 = t1.rename(columns = {'premise':'P','hypothesis':'H','label':'L'})
t1.L = t1.L.replace({"entailment":"e","neutral":"n","contradiction":"c"})
t1['D'] = "LingNLI"

t2 = pandas.read_json("https://raw.githubusercontent.com/Alicia-Parrish/ling_in_loop/master/NLI_data/3_Ling_in_loop_protocol/val_round5_LitL_combined.jsonl", lines=True)[['premise','hypothesis','label']]
t2 = t2.rename(columns = {'premise':'P','hypothesis':'H','label':'L'})
t2.L = t2.L.replace({"entailment":"e","neutral":"n","contradiction":"c"})
t2['D'] = "LingNLI"

print(len(t1)), print(len(t2))

t1 = pd.DataFrame(t1)

x = t1[t1['L'] == "e"]
train = pd.concat([t1, x], ignore_index=True)

t2 = pd.DataFrame(t2)

append_to_global(train, t2, None)

train.value_counts('L')

14995
2425


L
e    9998
n    4999
c    4997
Name: count, dtype: int64

In [11]:
#ConTRoL-NLI

# t1 = pandas.read_json("https://github.com/csitfun/ConTRoL-dataset/raw/main/data/train.jsonl", lines=True)[['premise','hypothesis','label']]
# t2 = pandas.read_json("https://github.com/csitfun/ConTRoL-dataset/raw/main/data/dev.jsonl", lines=True)[['premise','hypothesis','label']]
# t3 = pandas.read_json("https://github.com/csitfun/ConTRoL-dataset/raw/main/data/test.jsonl", lines=True)[['premise','hypothesis','label']]

# t1['D'] = "ConTRoL"
# t2['D'] = "ConTRoL"
# t3['D'] = "ConTRoL"

# t1 = t1.rename(columns = {'premise':'P','hypothesis':'H','label':'L'})
# t2 = t2.rename(columns = {'premise':'P','hypothesis':'H','label':'L'})
# t3 = t3.rename(columns = {'premise':'P','hypothesis':'H','label':'L'})

# t1 = pd.DataFrame(t1)
# t2 = pd.DataFrame(t2)
# t3 = pd.DataFrame(t3)

# print(len(t1)), print(len(t2)), print(len(t3))

# def get_length(row):
#     text_a = row['P']
#     text_a = len(text_a.split())
    
#     row['len(P)'] = text_a
    
#     return row

# t1 = t1.apply(get_length, axis=1)
# t2 = t2.apply(get_length, axis=1)
# t3 = t3.apply(get_length, axis=1)

# t1 = t1[t1["len(P)"] > 25]
# t2 = t2[t2["len(P)"] > 25]
# t3 = t3[t3["len(P)"] > 25]

# print(len(t1)), print(len(t2)), print(len(t3))

# append_to_global(t1, t2, t3)

In [12]:
# # Contract-NLI https://huggingface.co/datasets/kiddothe2b/contract-nli

# t1,t2,t3 = load_hf_dataset("kiddothe2b/contract-nli",
#                     ["train"],
#                     ["validation"],
#                     ["test"],
#                     ['premise','hypothesis','label'],
#                     {'1': 'e', '2': 'n', '0': 'c'},
#                     "contractnli_b"
#                    )

# #for each premise, we choose one hypothesis from the same label
# t1 = t1.drop_duplicates(subset=["P","L"], keep="first")

# tmp1 = t1[t1["L"] == "e"].sample(n=250, random_state=seed)
# tmp2 = t1[t1["L"] != "e"].sample(n=150, random_state=seed)

# train = pd.concat([tmp1, tmp2])

# t2 = t2.drop_duplicates(subset=["P","L"], keep="first")

# append_to_global(train, t2, None)

In [13]:
#VitaminC

dataset = load_dataset("tals/vitaminc")


tmp1 = dataset["train"].to_pandas()
tmp2 = dataset["validation"].to_pandas()
tmp3 = dataset["test"].to_pandas()

t1 = pd.DataFrame(tmp1)
t2 = pd.DataFrame(tmp2)
t3 = pd.DataFrame(tmp3)


t1 = t1.drop_duplicates(subset=["evidence"], keep="first")

#check if evidence has a number in it
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)


count = 0
def clean_vitaminc(row):
    premise = row['evidence']
    hypothesis = row['claim']
    label = row['label']
    page = row['page']
    premise_length = len(premise.split())
    hypothesis_length = len(hypothesis.split())
    mean = (premise_length + hypothesis_length) / 2
    
    if label == "SUPPORTS":
        label = "e"
    elif label == "REFUTES":
        label = "c"
    else:
        label = "n"
        
    row['P'] = f"Page Name: {page}\n\n{premise}"
    row['H'] = hypothesis
    row['L'] = label
    row['len'] = mean
    row['D'] = "vitaminc"
    row['hasNumbers'] = hasNumbers(hypothesis)
    
    global count
    if not count % 1000:
        print(count)
    count+=1
    
    return row

t1 = t1.apply(clean_vitaminc, axis=1)[["P","H","L","D", "len", "hasNumbers"]]
t1 = t1[t1['hasNumbers'] == True]

t1_ = t1[t1['len'] <= 35][["P","H","L","D"]]
t1_e = t1_[t1_['L'] == "e"].sample(n=500, random_state=seed)
t1_c = t1_[t1_['L'] == "c"].sample(n=800, random_state=seed)
t1_n = t1_[t1_['L'] == "n"].sample(n=800, random_state=seed)

t1 = t1[t1['len'] > 35][["P","H","L","D"]]
t1 = pd.concat([t1, t1_e, t1_c, t1_n], ignore_index=True)

t2 = t2.sample(n=1000, random_state=seed)
t2 = t2.apply(clean_vitaminc, axis=1)[["P","H","L","D"]]
# t3 = t3.apply(clean_vitaminc, axis=1)[["P","H","L","D"]]

append_to_global(t1, t2, None)

t1.value_counts('L')

(_deploy_ray_func pid=17470) 0
(_deploy_ray_func pid=17465) 3000 [repeated 24x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_deploy_ray_func pid=17471) 6000 [repeated 27x across cluster]
(_deploy_ray_func pid=17471) 9000 [repeated 23x across cluster]
(_deploy_ray_func pid=17471) 12000 [repeated 24x across cluster]
(_deploy_ray_func pid=17470) 15000 [repeated 23x across cluster]
(_deploy_ray_func pid=17466) 18000 [repeated 22x across cluster]
(_deploy_ray_func pid=17471) 22000 [repeated 18x across cluster]
(_deploy_ray_func pid=17468) 25000 [repeated 17x across cluster]
(_deploy_ray_func pid=17470) 0 [repeated 11x across cluster]


L
e    4206
n    2173
c    2110
Name: count, dtype: int64

In [14]:
# #HealthFact
# t1,t2,t3 = load_hf_dataset("health_fact",
#                     ["train"],
#                     ["validation"],
#                     ["test"],
#                     ['main_text','claim','label'],
#                     {'2': 'e', '1': 'n', '0': 'c'},
#                    )

# #remove -1 and 3
# t1 = t1[(t1['L'] != "-1") & (t1['L'] != "3")]
# t2 = t2[(t2['L'] != "-1") & (t2['L'] != "3")]
# t3 = t3[(t3['L'] != "-1") & (t3['L'] != "3")]

# count = 0
# def get_length(row):
#     global count
#     text_a = row['P']
#     text_a = len(text_a.split())
#     text_b = row['H']
#     text_b = len(text_b.split())
    
#     row['len(P)'] = text_a
#     row['len(H)'] = text_b
    
#     if not count % 1000:
#         print(count)
#     count+=1
    
#     return row

# t1 = t1.apply(get_length, axis=1)
# t1 = t1[t1['L'] != 'n']
# t1 = t1[t1["len(P)"] < 1500][["P","H","L","D", "len(H)"]]
# t1 = t1[t1['len(H)'] > 15][["P","H","L","D"]]

# tmp1 = t1[t1["L"] == "e"]
# tmp2 = t1[t1["L"] == "c"].sample(n=len(tmp1), random_state=seed)

# train = pd.concat([tmp1, tmp2])

# t2 = t2.apply(get_length, axis=1)
# t2 = t2[t2['L'] != 'n']
# t2 = t2[t2["len(P)"] < 1500][["P","H","L","D", "len(H)"]]
# t2 = t2[t2['len(H)'] > 15][["P","H","L","D"]].sample(n=200, random_state=seed)



# # t3 = t3.apply(get_length, axis=1)
# # t3 = t3[t3["len(P)"] < 2500][["P","H","L","D"]]

# append_to_global(train, t2, None)

In [15]:
# #HealthVer

# train = pandas.read_csv("https://raw.githubusercontent.com/sarrouti/HealthVer/master/data/healthver_train.csv")[['evidence','claim','label']]
# validation = pandas.read_csv("https://raw.githubusercontent.com/sarrouti/HealthVer/master/data/healthver_dev.csv")[['evidence','claim','label']]
# test = pandas.read_csv("https://raw.githubusercontent.com/sarrouti/HealthVer/master/data/healthver_test.csv")[['evidence','claim','label']]

# train['label'] = train['label'].astype(str).replace({'Supports': 'e', 'Neutral': 'n', 'Refutes': 'c'})
# validation['label'] = validation['label'].astype(str).replace({'Supports': 'e', 'Neutral': 'n', 'Refutes': 'c'})
# test['label'] = test['label'].astype(str).replace({'Supports': 'e', 'Neutral': 'n', 'Refutes': 'c'})


# train = train.rename(columns = {'evidence':'P','claim':'H','label':'L'})
# validation = validation.rename(columns = {'evidence':'P','claim':'H','label':'L'})
# test = test.rename(columns = {'evidence':'P','claim':'H','label':'L'})

# train['D'] = "HealthVer"
# validation['D'] = "HealthVer"
# test['D'] = "HealthVer"

# append_to_global(train, validation, test)

In [16]:
#DeFacto

train = pandas.read_json("https://github.com/microsoft/DeFacto/raw/main/data/train.jsonl", lines=True)[['article', 'candidate', 'has_error', 'feedback']]
validation = pandas.read_json("https://github.com/microsoft/DeFacto/raw/main/data/val.jsonl", lines=True)[['article', 'candidate', 'has_error', 'feedback']]
test = pandas.read_json("https://github.com/microsoft/DeFacto/raw/main/data/test.jsonl", lines=True)[['article', 'candidate', 'has_error', 'feedback']]

train = pd.DataFrame(train)
validation = pd.DataFrame(validation)
test = pd.DataFrame(test)

print(f"len(train): {len(train)}, len(validation): {len(validation)}, len(test): {len(test)}")

def clean_defacto(row):
    article = row['article']
    candidate = row['candidate']
    has_error = row['has_error']
    feedback = row['feedback']
    
    if has_error:
        label = "c"
        correct = feedback['summary']
    else:
        label = "e"
        correct = ""
    
    row['L'] = label
    row['D'] = "DeFacto"
    row['correct'] = correct
    
    return row


# Train set preprocessing
train = train.apply(clean_defacto, axis=1)

corrected_train = train[train['correct'] != ""]
uncorrected_train = train[train['correct'] == ""]

corrected_train_entailment = corrected_train.rename(columns = {'article':'P','correct':'H'})[['P','H','L','D']]
corrected_train_entailment['L'] = "e"
corrected_train_contradiction = corrected_train.rename(columns = {'article':'P','candidate':'H'})[['P','H','L','D']]
corrected_train_contradiction['L'] = "c"

corrected_train_contradiction_upsampled = corrected_train_contradiction.sample(300, random_state=seed)

uncorrected_train = uncorrected_train.rename(columns = {'article':'P','candidate':'H'})[['P','H','L','D']]
uncorrected_train['L'] = "e"

# Validation set preprocessing
validation = validation.apply(clean_defacto, axis=1)

corrected_validation = validation[validation['correct'] != ""]
uncorrected_validation = validation[validation['correct'] == ""]

corrected_validation_entailment = corrected_validation.rename(columns = {'article':'P','correct':'H'})[['P','H','L','D']]
corrected_validation_entailment['L'] = "e"
corrected_validation_contradiction = corrected_validation.rename(columns = {'article':'P','candidate':'H'})[['P','H','L','D']]
corrected_validation_contradiction['L'] = "c"

uncorrected_validation = uncorrected_validation.rename(columns = {'article':'P','candidate':'H'})[['P','H','L','D']]
uncorrected_validation['L'] = "e"

# Test set preprocessing
test = test.apply(clean_defacto, axis=1)

corrected_test = test[test['correct'] != ""]
uncorrected_test = test[test['correct'] == ""]
corrected_test_entailment = corrected_test.rename(columns = {'article':'P','correct':'H'})[['P','H','L','D']]
corrected_test_entailment['L'] = "e"
corrected_test_contradiction = corrected_test.rename(columns = {'article':'P','candidate':'H'})[['P','H','L','D']]
corrected_test_contradiction['L'] = "c"

uncorrected_test = uncorrected_test.rename(columns = {'article':'P','candidate':'H'})[['P','H','L','D']]
uncorrected_test['L'] = "e"

train = pd.concat([corrected_train_entailment, corrected_train_contradiction, uncorrected_train, corrected_train_contradiction_upsampled], ignore_index=True)
validation = pd.concat([corrected_validation_entailment, corrected_validation_contradiction, uncorrected_validation], ignore_index=True)
test = pd.concat([corrected_test_entailment, corrected_test_contradiction, uncorrected_test], ignore_index=True)

print(f"len(train): {len(train)}, len(validation): {len(validation)}, len(test): {len(test)}")

append_to_global(train, validation, test)

train.value_counts('L')

len(train): 1000, len(validation): 486, len(test): 1075
len(train): 2001, len(validation): 827, len(test): 1854


L
c    1001
e    1000
Name: count, dtype: int64

In [17]:
#DADC-NLI

train = pandas.read_json("https://raw.githubusercontent.com/facebookresearch/dadc-limit/main/data/dynamic-adversarial-with-rounds.jsonl", lines=True)
test = pandas.read_json("https://raw.githubusercontent.com/facebookresearch/dadc-limit/main/data/expert-test-set.jsonl", lines=True)

print(f"len(train): {len(train)}, len(test): {len(test)}")

train = pd.DataFrame(train)[['sentence1', 'sentence2', 'label']]
test = pd.DataFrame(test)[['sentence1', 'sentence2', 'label']]

#Rename
train = train.rename(columns = {'sentence1':'P','sentence2':'H','label':'L'})
test = test.rename(columns = {'sentence1':'P','sentence2':'H','label':'L'})

train['L'] = train['L'].astype(str).replace({'entailment': 'e', 'contradiction': 'c'})
test['L'] = test['L'].astype(str).replace({'entailment': 'e', 'contradiction': 'c'})

train_upsampled = train[train['L'] == "e"].sample(700, random_state=seed)
train = pd.concat([train, train_upsampled], ignore_index=True)
train['D'] = "DADC-NLI"
test['D'] = "DADC-NLI"

append_to_global(train, None, test)

train.value_counts('L')

len(train): 8665, len(test): 766


L
c    4693
e    4672
Name: count, dtype: int64

In [18]:
#FoolMeTwice

train = pandas.read_json("https://raw.githubusercontent.com/google-research/fool-me-twice/main/dataset/train.jsonl", lines=True)
train = pd.DataFrame(train)

validation = pandas.read_json("https://raw.githubusercontent.com/google-research/fool-me-twice/main/dataset/dev.jsonl", lines=True)
validation = pd.DataFrame(validation)

test = pandas.read_json("https://raw.githubusercontent.com/google-research/fool-me-twice/main/dataset/test.jsonl", lines=True)
test = pd.DataFrame(test)


print(f"len(train): {len(train)}, len(validation): {len(validation)}, len(test): {len(test)}")

def process_foolmetwice(row):
    page = row['wikipedia_page']
    text = row['text']
    gold_evidence = row['gold_evidence']
    
    joined_evidence = f"Page Name: {page}\n\n"
    for evidence in gold_evidence:
        section_header = evidence['section_header']
        joined_evidence += section_header + ": "
        joined_evidence += evidence['text'] + "\n"
    
    joined_evidence = joined_evidence.strip("\n")

    row['P'] = joined_evidence
    row['H'] = text
    row['L'] = row['label']
    row['D'] = "FoolMeTwice"
    
    return row

train = train.apply(process_foolmetwice, axis=1)[["P","H","L","D"]]
train['L'] = train['L'].astype(str).replace({'SUPPORTS': 'e', 'REFUTES': 'c'})
train_upsampled = train[train['L'] == "e"].sample(150, random_state=seed)
train = pd.concat([train, train_upsampled], ignore_index=True)

validation = validation.apply(process_foolmetwice, axis=1)[["P","H","L","D"]]
validation['L'] = validation['L'].astype(str).replace({'SUPPORTS': 'e', 'REFUTES': 'c'})

test = test.apply(process_foolmetwice, axis=1)[["P","H","L","D"]]
test['L'] = test['L'].astype(str).replace({'SUPPORTS': 'e', 'REFUTES': 'c'})

append_to_global(train, validation, test)

train.value_counts('L')

len(train): 10419, len(validation): 1169, len(test): 1380


L
c    5293
e    5276
Name: count, dtype: int64

In [19]:
# #CORE Dataset

# train = pandas.read_csv("https://raw.githubusercontent.com/Sai90000/ScientificHypothesisEvidencing/main/data/train_mapped.csv")[['abstract_collected', 'declarative', 'label']]
# validation = pandas.read_csv("https://raw.githubusercontent.com/Sai90000/ScientificHypothesisEvidencing/main/data/held_out_mapped.csv")[['abstract_collected', 'declarative', 'label']]
# test = pandas.read_csv("https://raw.githubusercontent.com/Sai90000/ScientificHypothesisEvidencing/main/data/test_mapped.csv")[['abstract_collected', 'declarative', 'label']]

# print(f"len(train): {len(train)}, len(validation): {len(validation)}, len(test): {len(test)}")

# train = pd.DataFrame(train)
# train['declarative'] = train['declarative'].apply(lambda x: str.lower(x))
# train = train.rename(columns = {'abstract_collected':'P','declarative':'H','label':'L'})
# train['L'] = train['L'].astype(str).replace({'ENTAIL': 'e', 'CONTRADICT': 'c', 'INCONCLUSIVE': 'n'})
# train['D'] = "CORE"

# validation = pd.DataFrame(validation)
# validation['declarative'] = validation['declarative'].apply(lambda x: str.lower(x))
# validation = validation.rename(columns = {'abstract_collected':'P','declarative':'H','label':'L'})
# validation['L'] = validation['L'].astype(str).replace({'ENTAIL': 'e', 'CONTRADICT': 'c', 'INCONCLUSIVE': 'n'})
# validation['D'] = "CORE"

# test = pd.DataFrame(test)
# test['declarative'] = test['declarative'].apply(lambda x: str.lower(x))
# test = test.rename(columns = {'abstract_collected':'P','declarative':'H','label':'L'})
# test['L'] = test['L'].astype(str).replace({'ENTAIL': 'e', 'CONTRADICT': 'c', 'INCONCLUSIVE': 'n'})
# test['D'] = "CORE"

# append_to_global(train, validation, test)

In [20]:
# #CoVERT Dataset: https://www.ims.uni-stuttgart.de/forschung/ressourcen/korpora/bioclaim/

# train = pandas.read_json("../data/covert/CoVERT_FC_annotations.jsonl", lines=True)
# train = pd.DataFrame(train)


# def get_covert(row):
#     claim = row['claim']
#     evidences = row['evidence']
#     pairs = []
#     for evidence in evidences:
#         if evidence[1]:
#             x = (claim, evidence[2], evidence[0])
#             # print(x)
#             pairs.append(x)

#     row['pairs'] = pairs
            
#     return row


# train = train.apply(get_covert, axis=1)
# pairs = train['pairs'].to_list()

# import re
# url_regex = re.compile(r'https?://\S+|www\.\S+')

# def clean_covert_text(text):
#     text = fix_text(text)
#     text = text.replace("@username", " ")
#     text = text.replace("[NEWLINE]", "\n")
#     text = re.sub(url_regex, '', text)
#     text = text.replace("Source:", " ")
#     return " ".join(text.split())

# clean_pairs = []
# for pair in pairs:
#     for p in pair:
#         claim = clean_covert_text(p[0])
#         evidence = p[1]
#         label = p[2]
#         clean_pairs.append((evidence, claim, label))
        

# train = pd.DataFrame(clean_pairs, columns=['P', 'H', 'L'])
# train["L"] = train["L"].replace({"SUPPORTS": "e", "REFUTES": "c", "NOT ENOUGH INFO": "n"})
# train['D'] = "CoVERT"
# print(f"len(train): {len(train)}")
# print(train['L'].value_counts())

# train = train.dropna()

# append_to_global(train, None, None)

In [21]:
#WiCE

train = pandas.read_json("https://raw.githubusercontent.com/ryokamoi/wice/main/data/entailment_retrieval/claim/train.jsonl",
                         lines=True)

validation = pandas.read_json("https://raw.githubusercontent.com/ryokamoi/wice/main/data/entailment_retrieval/claim/dev.jsonl",
                            lines=True)

test = pandas.read_json("https://raw.githubusercontent.com/ryokamoi/wice/main/data/entailment_retrieval/claim/test.jsonl",
                            lines=True)

train = pd.DataFrame(train)
validation = pd.DataFrame(validation)
test = pd.DataFrame(test)


def clean_wice(row):
    label = row['label']
    supporting_sentences = row['supporting_sentences']
    claim = row['claim']
    evidence = row['evidence']
    claim_title = row['meta']['claim_title']
    claim_section = row['meta']['claim_section']
    
    claim_meta = ""
    if claim_title:
        claim_meta = claim_meta + " " + claim_title
    if claim_section:
        claim_meta = claim_meta + " " + claim_section
    if claim_meta:
        claim = claim_meta + "\n" + claim
    
    all_supporting_sentences = []
    if "(meta data) TITLE:" in evidence[0]:
        evidence[0] = evidence[0].replace("(meta data) TITLE:", " ") + "\n"
        all_supporting_sentences.extend([0])
    for supporting_sentence in supporting_sentences:
        all_supporting_sentences.extend(supporting_sentence)
    all_supporting_sentences = list(set(all_supporting_sentences))
    all_supporting_sentences.sort()
    all_supporting_sentences = [evidence[sent] for sent in all_supporting_sentences]
    all_supporting_sentences = " ".join(all_supporting_sentences).strip()

    row['final_evidence'] = fix_text(all_supporting_sentences.strip())
    row['claim'] = fix_text(claim.strip())
    
    return row

train = train.apply(clean_wice, axis=1)[['final_evidence', 'claim', 'label']]
validation = validation.apply(clean_wice, axis=1)[['final_evidence', 'claim', 'label']]
test = test.apply(clean_wice, axis=1)[['final_evidence', 'claim', 'label']]

train = train.rename(columns = {'final_evidence':'P','claim':'H','label':'L'})
validation = validation.rename(columns = {'final_evidence':'P','claim':'H','label':'L'})
test = test.rename(columns = {'final_evidence':'P','claim':'H','label':'L'})

train["D"] = "WiCE"
validation["D"] = "WiCE"
test["D"] = "WiCE"

train['L'] = train['L'].astype(str).replace({'supported': 'e', 'not_supported': 'c', "partially_supported": "n"})
validation['L'] = validation['L'].astype(str).replace({'supported': 'e', 'not_supported': 'c', "partially_supported": "n"})
test['L'] = test['L'].astype(str).replace({'supported': 'e', 'not_supported': 'c', "partially_supported": "n"})

train_upsampled = train[train['L'] == "e"].sample(340, random_state=seed)
train = pd.concat([train, train_upsampled], ignore_index=True)

print(f"len(train): {len(train)}, len(validation): {len(validation)}, len(test): {len(test)}")

append_to_global(train, validation, test)

train.value_counts('L')

len(train): 1600, len(validation): 349, len(test): 358


L
e    800
n    633
c    167
Name: count, dtype: int64

In [22]:
# Seahorse

folder_path = "../data/seahorse_data/"

print('Preparing Seahorse data...')
train = pandas.read_json(folder_path + 'seahorse-train.jsonl', lines=True)
validation = pandas.read_json(folder_path + 'seahorse-validation.jsonl', lines=True)
test = pandas.read_json(folder_path + 'seahorse-test.jsonl', lines=True)

print('Formatting text...')
# train = train.apply(format_text, axis=1)[['text']]
# validation = validation.apply(format_text, axis=1)[['text']]
train = train.rename(columns = {'source':'P','target':'H','label':'L'})
validation = validation.rename(columns = {'source':'P','target':'H','label':'L'})
test = test.rename(columns = {'source':'P','target':'H','label':'L'})

train['L'] = train['L'].astype(str).replace({'Yes': 'e', 'No': 'c'})
validation['L'] = validation['L'].astype(str).replace({'Yes': 'e', 'No': 'c'})
test['L'] = test['L'].astype(str).replace({'Yes': 'e', 'No': 'c'})

train['D'] = "Seahorse"
validation['D'] = "Seahorse"
test['D'] = "Seahorse"

train_upsampled = train[train['L'] == "c"].sample(1780, random_state=seed)
train = pd.concat([train, train_upsampled], ignore_index=True)

# Double the number of examples in the training set 
train = pd.concat([train, train], ignore_index=True)


print(f"len(train): {len(train)}, len(validation): {len(validation)}, len(test): {len(test)}")

append_to_global(train, validation, test)

train.value_counts('L')

Preparing Seahorse data...
Formatting text...
len(train): 31666, len(validation): 2074, len(test): 4138


L
e    15838
c    15828
Name: count, dtype: int64

In [23]:
# BoolQ

boolq = pd.read_csv("boolq_positives.csv", sep=",")
boolq_dict = boolq.set_index('input').T.to_dict()
boolq_ds = load_dataset("boolq")


rows = []

for example in boolq_ds['train']:
    input_ = example['question']
    try:
        output = boolq_dict[input_]['output']
        passage = example['passage']
        label = example['answer']
        rows.append((passage, output, label))
    except Exception as e:
        continue
    

boolq_df = pd.DataFrame(rows, columns=['P', 'H', 'L'])
boolq_df['D'] = "boolq"
boolq_df['L'] = boolq_df['L'].astype(str).replace({'True': 'e', 'False': 'c'})

boolq_df_upsampled = boolq_df[boolq_df['L'] == "c"].sample(2300, random_state=seed)
boolq_df = pd.concat([boolq_df, boolq_df_upsampled], ignore_index=True)

append_to_global(boolq_df, None, None)

boolq_df.value_counts('L')

L
e    5873
c    5852
Name: count, dtype: int64

In [24]:
# all_train = all_train.drop_duplicates(subset=['P','H'], keep="first")
all_val = all_val.drop_duplicates(subset=['P','H'], keep="first")
all_test = all_test.drop_duplicates(subset=['P','H'], keep="first")

len(all_train), len(all_val), len(all_test)

(347804, 12618, 18815)

In [25]:
count = 0
def get_length(row):
    global count
    text_a = row['P']
    text_b = row['H']
    text_a = len(text_a.split())
    text_b = len(text_b.split())
    
    row['len(P)'] = text_a
    row['len(H)'] = text_b
    
    if not count % 1000:
        print(count)
    count+=1
    
    return row

In [26]:
from ftfy import fix_text

def fix_all(row):
    row['P'] = fix_text(row['P'])
    row['H'] = fix_text(row['H'])
    return row

all_train = all_train.apply(fix_all, axis=1)
all_val = all_val.apply(fix_all, axis=1)
all_test = all_test.apply(fix_all, axis=1)

In [27]:
all_train = all_train.apply(get_length, axis=1)
all_val = all_val.apply(get_length, axis=1)
all_test = all_test.apply(get_length, axis=1)

(_deploy_ray_func pid=17468) 0
(_deploy_ray_func pid=17468) 8000 [repeated 65x across cluster]
(_deploy_ray_func pid=17465) 16000 [repeated 60x across cluster]
(_deploy_ray_func pid=17470) 24000 [repeated 58x across cluster]
(_deploy_ray_func pid=17470) 32000 [repeated 55x across cluster]
(_deploy_ray_func pid=17472) 39000 [repeated 54x across cluster]
(_deploy_ray_func pid=17471) 47000 [repeated 33x across cluster]
(_deploy_ray_func pid=17466) 56000 [repeated 18x across cluster]
(_deploy_ray_func pid=17470) 0 [repeated 9x across cluster]


In [28]:
all_train.D.value_counts()

D
alisawuffles/WANLI    127885
anli                  105076
Seahorse               31666
LingNLI                19994
scitail                16944
boolq                  11725
FoolMeTwice            10569
vitaminc                8489
glue_rte                2490
DeFacto                 2001
WiCE                    1600
Name: count, dtype: int64

In [29]:
all_train.L.value_counts()

L
e    173006
c     97303
n     77495
Name: count, dtype: int64

In [30]:
all_train = all_train.rename(columns = {'P':'source','H':'target','L':'label','D':'subset'})
all_val = all_val.rename(columns = {'P':'source','H':'target','L':'label','D':'subset'})
all_test = all_test.rename(columns = {'P':'source','H':'target','L':'label','D':'subset'})

all_train['label'] = all_train['label'].astype(str).replace({'e': 'Yes', 'c': 'No', 'n': 'No'})
all_val['label'] = all_val['label'].astype(str).replace({'e': 'Yes', 'c': 'No', 'n': 'No'})
all_test['label'] = all_test['label'].astype(str).replace({'e': 'Yes', 'c': 'No', 'n': 'No'})

In [31]:
all_train['len(P)'].sum() + all_train['len(H)'].sum()

28404444

In [32]:
all_train.label.value_counts(), all_val.label.value_counts(), all_test.label.value_counts()

(label
 No     174798
 Yes    173006
 Name: count, dtype: int64,
 label
 No     6992
 Yes    5626
 Name: count, dtype: int64,
 label
 No     10601
 Yes     8214
 Name: count, dtype: int64)

In [33]:
out_columns = ['source', 'target', 'label', 'subset']

all_train[out_columns].to_json("../data/combined-train.jsonl", lines=True, orient="records")
all_val[out_columns].to_json("../data/combined-validation.jsonl", lines=True, orient="records")
all_test[out_columns].to_json("../data/combined-test.jsonl", lines=True, orient="records")

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [34]:
# all_train = pandas.read_json("../data/combined-train.jsonl", lines=True)

all_train.subset.value_counts()


subset
alisawuffles/WANLI    127885
anli                  105076
Seahorse               31666
LingNLI                19994
scitail                16944
boolq                  11725
FoolMeTwice            10569
vitaminc                8489
glue_rte                2490
DeFacto                 2001
WiCE                    1600
Name: count, dtype: int64

In [35]:
pd.options.display.max_colwidth = 2000

all_train[all_train.subset =="alisawuffles/WANLI"].sample(5)

,source,target,label,subset,len(P),len(H)
173289,You must be crazy to be trying to go up against the Bisons.,The Bisons are not good.,No,alisawuffles/WANLI,13,5
124093,"The wily fox, after escaping from the lion's den, took refuge in a tree.",The fox hid in the tree after he was captured.,No,alisawuffles/WANLI,14,10
134258,"In the early part of the nineteenth century, the Irish, or ""Famine Irish"" as they were called, emigrated in large numbers to the United States.",The Irish were the only people who emigrated to the United States.,No,alisawuffles/WANLI,25,12
179366,I've never seen any good movies.,Some movies are good.,No,alisawuffles/WANLI,6,4
201325,"If you think that being a newspaper reporter is hard, try being a newspaper reporter and a football player.","Football players are a lot of fun, but reporters are a lot of work.",No,alisawuffles/WANLI,19,14


In [36]:
# all_train[all_train['len(P)'] > 1500]

In [37]:
#print total len(P) and len(H) for each dataset

all_train.groupby('subset')['len(P)'].sum()

subset
DeFacto                 579057
FoolMeTwice             410330
LingNLI                 388897
Seahorse              11763554
WiCE                    123158
alisawuffles/WANLI     2240403
anli                   5720300
boolq                  1092152
glue_rte                108478
scitail                 301743
vitaminc                541884
Name: len(P), dtype: int64

In [38]:
len(all_train), len(all_val), len(all_test)

(347804, 12618, 18815)

In [39]:
pd.DataFrame(all_train.groupby('subset').label.value_counts())

count
subset             label       
DeFacto            No      1001
                   Yes     1000
FoolMeTwice        No      5293
                   Yes     5276
LingNLI            Yes     9998
                   No      9996
Seahorse           Yes    15838
                   No     15828
WiCE               Yes      800
                   No       800
alisawuffles/WANLI No     64374
                   Yes    63511
anli               No     52965
                   Yes    52111
boolq              Yes     5873
                   No      5852
glue_rte           Yes     1249
                   No      1241
scitail            Yes     8472
                   No      8472
vitaminc           No      4283
                   Yes     4206

In [40]:
pd.DataFrame(all_val.groupby('subset').label.value_counts())

count
subset      label       
DeFacto     Yes      486
            No       337
FoolMeTwice Yes      596
            No       573
LingNLI     No      1546
            Yes      876
Seahorse    Yes     1178
            No       896
WiCE        No       234
            Yes      115
anli        No      2130
            Yes     1070
glue_rte    Yes      146
            No       131
scitail     Yes      657
            No       647
vitaminc    Yes      502
            No       498

In [41]:
# Average token length of P and H for each subset

all_train.groupby('subset')['len(P)'].mean().sort_values()

subset
alisawuffles/WANLI     17.518888
scitail                17.808251
LingNLI                19.450685
FoolMeTwice            38.823919
glue_rte               43.565462
anli                   54.439644
vitaminc               63.833667
WiCE                   76.973750
boolq                  93.147292
DeFacto               289.383808
Seahorse              371.488473
Name: len(P), dtype: float64

In [42]:
all_train.groupby('subset')['len(H)'].mean().sort_values()

subset
boolq                  8.557783
glue_rte               8.790361
anli                   9.670572
alisawuffles/WANLI     9.850663
LingNLI               10.074322
scitail               11.687559
FoolMeTwice           13.685685
DeFacto               16.770115
vitaminc              19.551184
Seahorse              22.963431
WiCE                  27.805000
Name: len(H), dtype: float64

In [43]:
all_test.groupby('subset')['len(P)'].mean().sort_values()

subset
scitail                16.849953
alisawuffles/WANLI     17.483800
FoolMeTwice            39.286439
anli                   54.415625
WiCE                   88.329609
DeFacto               309.589286
Seahorse              383.268004
Name: len(P), dtype: float64

In [44]:
all_val.groupby('subset')['len(P)'].mean().sort_values()

subset
scitail         17.312883
LingNLI         19.819571
vitaminc        33.559000
FoolMeTwice     40.073567
glue_rte        42.386282
anli            54.464062
WiCE            90.879656
DeFacto        305.572296
Seahorse       369.850048
Name: len(P), dtype: float64

In [45]:
len(all_train), len(all_val), len(all_test)

(347804, 12618, 18815)

: 